In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .set("spark.sql.shuffle.partitions", "10")
 

sc = SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SQLContext
# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-32e00efb-ad69-460c-964d-d6d9c2b03bf6;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	

In [9]:
from ipyleaflet import Map, basemaps, Marker, Icon, Popup, Polyline
from ipywidgets import HTML
import random

colors = [
    # Colors used for the lines
    "red", "blue", "green", "yellow", "purple", "orange", 
    "pink", "brown", "black", "gray", "white", "cyan", "magenta"
]

def template_popup_plane(timestamp,country_code,velocity):
       return f"""
 <div style="font-family: Arial, sans-serif; color: #333; padding: 5px; border-radius: 5px;">
     <h3 style="margin: 0; color: #007BFF;">✈ Flying plane</h3>
     <p><b>Origin Country :</b> {country_code} </p>
     <p><b>Velocity :</b> {velocity if velocity != None else "Unavailable"} m/s</p>
     <p><b>Timestamp :</b> {timestamp}</p>
 </div>
"""

dict_plane_location = {}
def actualise_plane_position(batch_df, batch_id):
    data = batch_df.collect()
    new_markers = []  # Liste pour stocker les nouveaux marqueurs

    for row in data:
        # On récupère la position de l'avion
        location = [float(row["latitude"]), float(row["longitude"])]

        # Création du popup
        popup = HTML()
        popup.value = template_popup_plane(row["timestamp"], row["callsign"], row["velocity"])

        if location[0] != None and location[1] != None:
            if row["icao24"] not in dict_plane_location:
                # We create a marker because it is a new plane
                marker = Marker(location=location, icon=icon, draggable=False)
                marker.popup = popup

                # We store the marker in the dict and the polyline to draw its path
                polyline = Polyline(
                    locations=[location],
                    color=colors[random.randrange(0, len(colors))],
                    weight=1,
                    opacity=0.8,
                    fill=False
                )
                m.add(polyline)
                dict_plane_location[row["icao24"]] = {"marker":marker,"lines":polyline}
                m.add(marker)
            else:
                # The plane already exist
                marker = dict_plane_location[row["icao24"]]["marker"]
                marker.popup = popup
                marker.location = location

                polyline = dict_plane_location[row["icao24"]]["lines"]
                polyline.locations = polyline.locations + [location]

   
    print("Nombre d'avions mis à jour :", len(data))

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, avg
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType

# Kafka configuration
kafka_broker = "kafka1:9092"
kafka_topic = "planes"


In [ ]:
# Define schema for the Kafka message
schema = StructType([
    StructField("icao24", StringType(), True),
    StructField("callsign", StringType(), True),
    StructField("origin_country", StringType(), True),
    StructField("velocity", FloatType(), True),
    StructField("timestamp", StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True),
    StructField("category", StringType(), True)
])

# Read raw data from Kafka
raw_stream = sql_context.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

df_parsed = raw_stream.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

df_filtered = df_parsed.filter(col("origin_country") == "Germany")

# # Output rolling average to the console
query = df_filtered.writeStream \
    .foreachBatch(actualise_plane_position) \
    .outputMode("append") \
    .start()

# query_console = df_parsed.writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

query.awaitTermination()
# query_console.awaitTermination()

25/03/20 10:39:30 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-260976fb-0eae-434b-95a0-8b989bc3dacd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/20 10:39:30 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/20 10:39:30 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
            

KeyboardInterrupt: 

In [7]:
query.stop()
# query_console.stop()

In [ ]:
query.stop()

In [8]:
from ipyleaflet import Map, basemaps, Marker, Icon, Popup
from ipywidgets import HTML
import time

icon_url = "./avion_icon.png"
icon = Icon(icon_url=icon_url, icon_size=[20, 20]) 

center = [45.3288, 1.0]

m = Map(center=center, zoom=5)

colors = [
    # Noms de couleurs HTML courants
    "red", "blue", "green", "yellow", "purple", "orange", 
    "pink", "brown", "black", "gray", "white", "cyan", "magenta"
]

polyline = Polyline(
                locations=[center],
                color=colors[random.randrange(0, len(colors))],
                weight=3,
                opacity=0.8
            )

m.add(polyline)
# message2 = HTML()
# popup_html = f"""
# <div style="font-family: Arial, sans-serif; color: #333; padding: 5px; border-radius: 5px;">
#     <h3 style="margin: 0; color: #007BFF;">✈ Avion en vol</h3>
#     <p><b>Pays d'origine :</b> France </p>
#     <p><b>Vélocité :</b> 1.1 km/h</p>
#     <p><b>Timestamp :</b> 2024-12-15</p>
# </div>
# """
# message2.value = popup_html

# marker = Marker(location=center, icon=icon, draggable=False)
# marker.popup = message2

# m.add(marker)

m


Map(center=[45.3288, 1.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…